# Implementación de un Algoritmo para Buscaminas usando Análisis de Riesgo con Heuristica

## Introducción**

Este reporte describe la implementación de un algoritmo en Python para jugar automáticamente al Buscaminas. El algoritmo utiliza procesamiento de imágenes para leer el estado del juego y un análisis de riesgo para determinar la mejor jugada.

## Descripción del Problema

El Buscaminas es un juego de lógica donde el objetivo es descubrir todas las casillas del tablero que no contienen minas. La dificultad radica en que solo se conoce el número de minas adyacentes a una casilla descubierta.

**Implementación**

Nuestra implementación consta de tres partes principales: procesamiento de la imagen del juego, análisis del tablero, y toma de decisiones.

# Procesamiento de la Imagen

Utilizamos pyautogui para capturar la pantalla y opencv-python junto con easyocr para procesar la imagen y extraer la información del tablero.

In [39]:
import cv2
import numpy as np
import easyocr
import pyautogui
from typing import List, Tuple, Optional


In [40]:
SCREEN_WIDTH, SCREEN_HEIGHT = pyautogui.size()
GAME_REGION = (SCREEN_WIDTH // 2+15, SCREEN_HEIGHT // 6+20, 43 * SCREEN_WIDTH // 105 -21, 92 * SCREEN_HEIGHT // 125-15)
BOARD_SIZE = 9

In [41]:
def screenshot_gameboard():
    return pyautogui.screenshot(region=GAME_REGION)

In [42]:

def process_board_cells(gray_image, cell_size, reader):
    board_matrix = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            cell_img = gray_image[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            avg_pixel_value = np.mean(cell_img)
            if avg_pixel_value < 128:
                board_matrix[i, j] = -1
            cell_img_inverted = cv2.bitwise_not(cell_img)
            result = reader.readtext(cell_img_inverted, detail=0)
            text = result[0].strip() if result else ""
            if text.isdigit():
                board_matrix[i, j] = int(text)
            elif avg_pixel_value >= 128:
                board_matrix[i, j] = 0
    
    return board_matrix

In [43]:
def process_game_board():
    screenshot = screenshot_gameboard()
    screenshot.save("captura_color.png")
    screenshot_np = np.array(screenshot)
    gray_image = cv2.cvtColor(screenshot_np, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("captura_gris.png", gray_image)
    cell_size = 83  
    board_matrix = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
    reader = easyocr.Reader(['en'], gpu=False)
    board_matrix = process_board_cells(gray_image, cell_size, reader)
    return board_matrix

# Análisis del Tablero y Toma de Decisiones

Implementamos una función get_best_cell que analiza el tablero y determina la celda con menor riesgo de contener una mina.

In [45]:
def get_best_cell(board_matrix: List[List[int]]) -> Optional[Tuple[int, int]]:
    def is_valid(x: int, y: int) -> bool:
        return 0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE

    def get_unopened_neighbors(x: int, y: int) -> List[Tuple[int, int]]:
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                if is_valid(nx, ny) and board_matrix[nx][ny] == 0:
                    neighbors.append((nx, ny))
        return neighbors

    def get_number_neighbors(x: int, y: int) -> List[Tuple[int, int]]:
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                if is_valid(nx, ny) and board_matrix[nx][ny] > 0:
                    neighbors.append((nx, ny))
        return neighbors

    best_cell = None
    min_risk = float('inf')

    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if board_matrix[i][j] == 0:  # Unopened cell
                risk = 0
                number_neighbors = get_number_neighbors(i, j)
                for nx, ny in number_neighbors:
                    unopened_neighbors = get_unopened_neighbors(nx, ny)
                    if unopened_neighbors:
                        risk += board_matrix[nx][ny] / len(unopened_neighbors)
                
                if risk < min_risk:
                    min_risk = risk
                    best_cell = (i, j)

    return best_cell

**Bucle Principal del Juego**

La función solve_mine_sweeper implementa el bucle principal del juego, que repite el proceso de analizar el tablero, elegir la mejor jugada y realizarla.

In [46]:
def solve_mine_sweeper():
    board_matrix = process_game_board()
    while True:
        best_cell = get_best_cell(board_matrix)
        if best_cell is None:
            break
        i, j = best_cell
        pyautogui.click(GAME_REGION[0] + j * 83 + 41, GAME_REGION[1] + i * 83 + 41)
        board_matrix = process_game_board()

## Ejecución final

In [51]:
if __name__ == "__main__":
    solve_mine_sweeper()

Using CPU. Note: This module is much faster with a GPU.


# Resultados y Análisis

El algoritmo implementado utiliza un enfoque de análisis de riesgo para determinar la mejor jugada. Para cada celda no abierta, calcula un "riesgo" basado en la información de las celdas vecinas. Este enfoque permite al algoritmo tomar decisiones informadas incluso en situaciones donde no hay una jugada segura garantizada.

Ventajas del enfoque:

1. Adaptabilidad: El algoritmo puede ajustarse a diferentes estados del juego.
2. Eficiencia: Al considerar solo las celdas relevantes, el algoritmo puede tomar decisiones rápidamente.
3. Basado en probabilidades: El uso de un sistema de riesgo permite manejar la incertidumbre inherente al juego.

Limitaciones:

1. No garantiza una solución óptima: En algunos casos, el azar sigue jugando un papel importante.
2. Dependencia de la calidad de la imagen: El rendimiento del algoritmo está ligado a la precisión del procesamiento de la imagen.

**Conclusiones**

Hemos implementado con éxito un algoritmo capaz de jugar al Buscaminas de forma autónoma. El enfoque de análisis de riesgo proporciona una estrategia robusta para tomar decisiones en un entorno con información incompleta.
Posibles mejoras futuras incluyen:

Refinamiento del algoritmo de procesamiento de imágenes para mejorar la precisión.
Implementación de técnicas de aprendizaje automático para optimizar la estrategia de juego.
Extensión del algoritmo para manejar diferentes tamaños de tablero y niveles de dificultad.

**Referencias**

1. OpenCV Documentation. https://docs.opencv.org/

2. EasyOCR Documentation. https://github.com/JaidedAI/EasyOCR

3. PyAutoGUI Documentation. https://pyautogui.readthedocs.io/